In [ ]:
!pip install numpy matplotlib seaborn

In [ ]:
import csv
import dateutil
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import shutil
import urllib.request

In [ ]:
sns.set_theme()
%matplotlib inline

Download data from https://check-for-flooding.service.gov.uk/river-and-sea-levels?lng=-0.58408&lat=52.49929

https://riverlevels.uk/hebden-water-hebden-royd-nutclough

http://environment.data.gov.uk/flood-monitoring/id/stations/L1231/readings.csv?parameter=level&since=2010-01-01&_limit=10000&_sorted

In [ ]:
with urllib.request.urlopen('http://environment.data.gov.uk/flood-monitoring/id/stations/L1231/readings.csv?parameter=level&since=2010-01-01&_limit=10000&_sorted') as response:
    raw_data = [r.decode('utf-8') for r in response.readlines()]
    
    with open('L1231.csv', 'w') as csv_file:
        csv_file.write(raw_data[0])
        csv_file.write(''.join(raw_data[:1:-1]))


In [ ]:
# RIVER_DATA='Nutclough-height-data-20220201-1130.csv'
# RIVER_DATA='readings.csv'
RIVER_DATA='L1231.csv'

In [ ]:
with open(RIVER_DATA, 'r') as f:
    data = list(csv.reader(f, delimiter=","))

In [ ]:
dates = [dateutil.parser.isoparse(r[0]) for r in data[1:]]
levels = np.array([float(r[2]) for r in data[1:]])

In [ ]:
plt.figure(figsize = (8, 6))
plt.plot(dates, levels, 'r')     
plt.xticks(rotation=30, ha='right')
plt.xlabel('Date')
plt.ylabel('Level')
plt.show()

In [ ]:
ts = 15 * 60 # Sampling is 15 minutes
fs = 1.0 / ts

In [ ]:
fft_size=100
overlap_fac=0.8
hop_size = np.int32(np.floor(fft_size * (1-overlap_fac)))
pad_end_size = fft_size          # the last segment can overlap the end of the data array by no more than one window size
total_segments = np.int32(np.ceil(len(data) / np.float32(hop_size)))
t_max = len(levels) / np.float32(fs)
window = np.hanning(fft_size)  # our half cosine window
inner_pad = np.zeros(fft_size) # the zeros which will be used to double each segment size

In [ ]:
proc = np.concatenate((levels, np.zeros(pad_end_size)))              # the data to process
result = np.empty((total_segments, fft_size), dtype=np.float32)      # space to hold the result
for i in range(total_segments):                       # for each segment
    current_hop = hop_size * i                        # figure out the current segment offset
    segment = proc[current_hop:current_hop+fft_size]  # get the current segment
    windowed = segment * window                       # multiply by the half cosine function
    padded = np.append(windowed, inner_pad)           # add 0s to double the length of the data
    spectrum = np.fft.fft(padded) / fft_size          # take the Fourier Transform and scale by the number of samples
    autopower = np.abs(spectrum * np.conj(spectrum))  # find the autopower spectrum
    result[i, :] = autopower[:fft_size]               # append to the results array

result = 20*np.log10(result)          # scale to db
result = np.clip(result, -150, 200)    # clip values

In [ ]:
plt.figure(figsize = (8, 6))
plt.imshow(np.transpose(result), origin='lower', cmap='jet', interpolation='nearest', aspect='auto')
plt.ylabel('Frequency Bin')
plt.xlabel('Time')
plt.xticks([])
plt.show()

In [ ]:
t_max